In [43]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [44]:
data_dir = '/content/drive/MyDrive/DATASET/UCI HAR Dataset'

In [45]:
import csv
import numpy as np
import tensorflow as tf
import pandas as pd


In [46]:
# load a single file as a numpy array
def load_file(filepath):
	dataframe = pd.read_csv(filepath, header=None, delim_whitespace=True)
	return dataframe.values

We can then load all data for a given group (train or test) into a single three-dimensional NumPy array, where the dimensions of the array are [samples, time steps, features].

To make this clearer, there are 128 time steps and nine features, where the number of samples is the number of rows in any given raw signal data file.

The load_group() function below implements this behavior. The dstack() NumPy function allows us to stack each of the loaded 3D arrays into a single 3D array where the variables are separated on the third dimension (features).

In [47]:
# load a list of files into a 3D array of [samples, timesteps, features]
def load_group(filenames, prefix='/content/drive/MyDrive/DATASET/UCI HAR Dataset/'):
	loaded = list()
	for name in filenames:
		print('prefix + name : ', prefix + name)
		data = load_file(prefix + name)
		loaded.append(data)
	# stack group so that features are the 3rd dimension

	print('loaded.shape : ', len(loaded))
	loaded = np.dstack(loaded)
	print('loaded.shape : ', len(loaded))

	return loaded

In [48]:
prefix  = '/content/drive/MyDrive/DATASET/UCI HAR Dataset/'
group = 'train'


We can use this function to load all input signal data for a given group, such as train or test.

The load_dataset_group() function below loads all input signal data and the output data for a single group using the consistent naming conventions between the directories.

In [49]:
# load a dataset group, such as train or test
def load_dataset_group(group, prefix='/content/drive/MyDrive/DATASET/UCI HAR Dataset/'):
	filepath = prefix + group + '/Inertial Signals/'
	# load all 9 files as a single array
	filenames = list()
	# total acceleration
	filenames += ['total_acc_x_'+group+'.txt', 'total_acc_y_'+group+'.txt', 'total_acc_z_'+group+'.txt']
	# body acceleration
	filenames += ['body_acc_x_'+group+'.txt', 'body_acc_y_'+group+'.txt', 'body_acc_z_'+group+'.txt']
	# body gyroscope
	filenames += ['body_gyro_x_'+group+'.txt', 'body_gyro_y_'+group+'.txt', 'body_gyro_z_'+group+'.txt']
	# load input data
	print('filepath', filepath)
	X = load_group(filenames, filepath)
	# load class output
	y = load_file(prefix + group + '/y_'+group+'.txt')
	return X, y

Finally, we can load each of the train and test datasets.

The output data is defined as an integer for the class number. We must one hot encode these class integers so that the data is suitable for fitting a neural network multi-class classification model. We can do this by calling the to_categorical() Keras function.

The load_dataset() function below implements this behavior and returns the train and test X and y elements ready for fitting and evaluating the defined models.

In [50]:
# load the dataset, returns train and test X and y elements
def load_dataset(prefix='/content/drive/MyDrive/DATASET/UCI HAR Dataset/'):
	# load all train
	# trainX, trainy = load_dataset_group('train', prefix + 'HARDataset/')
	trainX, trainy = load_dataset_group('train', prefix)
	print(trainX.shape, trainy.shape)
	# load all test
	testX, testy = load_dataset_group('test', prefix)
 	# testX, testy = load_dataset_group('test', prefix + 'HARDataset/')
	print(testX.shape, testy.shape)
	# zero-offset class values
	trainy = trainy - 1
	testy = testy - 1
	# one hot encode y
	trainy = tf.keras.utils.to_categorical(trainy)
	testy = tf.keras.utils.to_categorical(testy)
	print('trainX.shape : ',trainX.shape,'\ntrainy.shape : ', trainy.shape, '\ntestX.shape : ',testX.shape ,'\ntestX.shape : ', testX.shape, '\ntesty.shape : ',testy.shape)
	return trainX, trainy, testX, testy

/content/drive/MyDrive/DATASET/UCI HAR Dataset/train/Inertial Signals/total_acc_x_train.txt

/content/drive/MyDrive/DATASET/UCI HAR DatasetHARDataset/train/Inertial Signals/total_acc_x_train.txt

In [51]:
trainX, trainy, testX, testy = load_dataset(prefix)

filepath /content/drive/MyDrive/DATASET/UCI HAR Dataset/train/Inertial Signals/
prefix + name :  /content/drive/MyDrive/DATASET/UCI HAR Dataset/train/Inertial Signals/total_acc_x_train.txt
prefix + name :  /content/drive/MyDrive/DATASET/UCI HAR Dataset/train/Inertial Signals/total_acc_y_train.txt
prefix + name :  /content/drive/MyDrive/DATASET/UCI HAR Dataset/train/Inertial Signals/total_acc_z_train.txt
prefix + name :  /content/drive/MyDrive/DATASET/UCI HAR Dataset/train/Inertial Signals/body_acc_x_train.txt
prefix + name :  /content/drive/MyDrive/DATASET/UCI HAR Dataset/train/Inertial Signals/body_acc_y_train.txt
prefix + name :  /content/drive/MyDrive/DATASET/UCI HAR Dataset/train/Inertial Signals/body_acc_z_train.txt
prefix + name :  /content/drive/MyDrive/DATASET/UCI HAR Dataset/train/Inertial Signals/body_gyro_x_train.txt
prefix + name :  /content/drive/MyDrive/DATASET/UCI HAR Dataset/train/Inertial Signals/body_gyro_y_train.txt
prefix + name :  /content/drive/MyDrive/DATASET/UCI

In [52]:
trainy


array([[0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1., 0.],
       ...,
       [0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.]], dtype=float32)

In [53]:
n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]

In [54]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout

In [55]:
n_timesteps

128

In [56]:
# model = Sequential()
# model.add(LSTM(100, input_shape=(n_timesteps,n_features)))
# model.add(Dropout(0.5))
# model.add(Dense(100, activation='relu'))
# model.add(Dense(n_outputs, activation='softmax'))
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [57]:
# fit and evaluate a model
def evaluate_model(trainX, trainy, testX, testy):
	verbose, epochs, batch_size = 1, 15, 64
	n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
	model = Sequential()
	model.add(LSTM(100, input_shape=(n_timesteps,n_features)))
	model.add(Dropout(0.5))
	model.add(Dense(100, activation='relu'))
	model.add(Dense(n_outputs, activation='softmax'))
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	# fit network
	model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
	# evaluate model
	_, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=1)
	return accuracy

In [58]:
# summarize scores
def summarize_results(scores):
	print(scores)
	m, s = np.mean(scores), np.std(scores)
	print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))

In [59]:
# run an experiment
def run_experiment(repeats=10):
	# load data
	trainX, trainy, testX, testy = load_dataset()
	# repeat experiment
	scores = list()
	for r in range(repeats):
		score = evaluate_model(trainX, trainy, testX, testy)
		score = score * 100.0
		print('>#%d: %.3f' % (r+1, score))
		scores.append(score)
	# summarize results
	summarize_results(scores)

In [ ]:
run_experiment()

filepath /content/drive/MyDrive/DATASET/UCI HAR Dataset/train/Inertial Signals/
prefix + name :  /content/drive/MyDrive/DATASET/UCI HAR Dataset/train/Inertial Signals/total_acc_x_train.txt
prefix + name :  /content/drive/MyDrive/DATASET/UCI HAR Dataset/train/Inertial Signals/total_acc_y_train.txt
prefix + name :  /content/drive/MyDrive/DATASET/UCI HAR Dataset/train/Inertial Signals/total_acc_z_train.txt
prefix + name :  /content/drive/MyDrive/DATASET/UCI HAR Dataset/train/Inertial Signals/body_acc_x_train.txt
prefix + name :  /content/drive/MyDrive/DATASET/UCI HAR Dataset/train/Inertial Signals/body_acc_y_train.txt
prefix + name :  /content/drive/MyDrive/DATASET/UCI HAR Dataset/train/Inertial Signals/body_acc_z_train.txt
prefix + name :  /content/drive/MyDrive/DATASET/UCI HAR Dataset/train/Inertial Signals/body_gyro_x_train.txt
prefix + name :  /content/drive/MyDrive/DATASET/UCI HAR Dataset/train/Inertial Signals/body_gyro_y_train.txt
prefix + name :  /content/drive/MyDrive/DATASET/UCI